# Análise de Mercado Financeiro

## Relatório Executivo de Investimentos

Este notebook realiza uma análise completa dos dados de mercado financeiro armazenados no PostgreSQL.

**Objetivo**: Identificar os principais indicadores de risco e oportunidade no mercado de ações.

---

## Passo 1: Importar Bibliotecas

Vamos importar as ferramentas necessárias para análise e visualização de dados.

In [ ]:
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configurar estilo dos gráficos
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Criar pasta para salvar resultados
output_dir = Path('outputs')
output_dir.mkdir(exist_ok=True)

print("Bibliotecas importadas com sucesso!")
print("Pasta 'outputs' criada para salvar resultados")

## Passo 2: Conectar ao Data Warehouse

Vamos conectar ao PostgreSQL e carregar os dados das ações.

In [ ]:
# Conectar ao banco de dados
print("🔌 Conectando ao PostgreSQL...")
conn = psycopg2.connect(
    dbname="financial_dw",
    user="airflow",
    password="airflow",
    host="localhost",
    port="5432"
)

# Consulta para carregar dados
query = """
    SELECT
        data_id,
        ticker,
        close as preco_fechamento,
        variacao_diaria,
        volume
    FROM fact_movimentacao_diaria
    ORDER BY data_id DESC
"""

# Carregar dados
df = pd.read_sql_query(query, conn)
conn.close()

print("✅ Dados carregados com sucesso!")
print(f"📊 Total de registros: {len(df):,}")
print(f"🏷️  Número de ações: {df['ticker'].nunique()}")
print(f"📅 Período: {df['data_id'].min()} até {df['data_id'].max()}")

---

# 📈 ANÁLISE 1: VOLATILIDADE DAS AÇÕES

## O que é Volatilidade?

A volatilidade mede **o quanto o preço de uma ação varia**. Ações com alta volatilidade têm preços que oscilam muito, indicando:
- ✅ **Maior potencial de ganho**
- ⚠️ **Maior risco de perda**

Calculamos a volatilidade usando o **desvio padrão** da variação diária.

## Cálculo: Top 5 Ações Mais Voláteis

In [ ]:
# Calcular volatilidade para cada ação
volatilidade = df.groupby('ticker')['variacao_diaria'].std()

# Selecionar as 5 ações mais voláteis
top_5_volateis = volatilidade.nlargest(5)

# Exibir resultados
print("🏆 TOP 5 AÇÕES MAIS VOLÁTEIS")
print("=" * 40)
for posicao, (acao, valor) in enumerate(top_5_volateis.items(), 1):
    print(f"{posicao}. {acao}: {valor:.2f}% de volatilidade")

print("\n💡 Dica: Ações voláteis oferecem maior risco, mas também maior potencial de retorno!")

## Gráfico: Visualizando a Volatilidade

In [ ]:
# Preparar dados para o gráfico
dados_grafico = top_5_volateis.reset_index()
dados_grafico.columns = ['Ação', 'Volatilidade']

# Criar figura
plt.figure(figsize=(12, 6))
barras = plt.bar(dados_grafico['Ação'], dados_grafico['Volatilidade'], 
                  color='#FF6B6B', alpha=0.8, edgecolor='black', linewidth=2)

# Adicionar valores nas barras
for barra in barras:
    altura = barra.get_height()
    plt.text(barra.get_x() + barra.get_width()/2., altura,
             f'{altura:.1f}%',
             ha='center', va='bottom', fontsize=11, fontweight='bold')

# Personalizar gráfico
plt.title('Top 5 Ações com Maior Volatilidade', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Código da Ação', fontsize=12, fontweight='bold')
plt.ylabel('Volatilidade (%)', fontsize=12, fontweight='bold')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()

# Salvar figura
plt.savefig(output_dir / '01_volatilidade.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráfico salvo em: outputs/01_volatilidade.png")

---

# 📈 ANÁLISE 2: VARIAÇÃO MÉDIA DE PREÇO

## O que é Variação Média?

A variação média mostra se uma ação **tende a subir ou descer** de preço ao longo do tempo:
- ✅ **Valores POSITIVOS** = preço tende a subir (boa performance)
- ❌ **Valores NEGATIVOS** = preço tende a descer (má performance)

Esta métrica ajuda a identificar ações com melhor desempenho histórico.

## Cálculo: Top 5 Ações com Melhor Performance

In [ ]:
# Calcular variação média para cada ação
variacao_media = df.groupby('ticker')['variacao_diaria'].mean()

# Selecionar as 5 ações com melhor performance
top_5_performance = variacao_media.nlargest(5)

# Exibir resultados
print("🏆 TOP 5 AÇÕES COM MELHOR PERFORMANCE")
print("=" * 45)
for posicao, (acao, valor) in enumerate(top_5_performance.items(), 1):
    print(f"{posicao}. {acao}: +{valor:.2f}% de variação média diária")

print("\n💡 Dica: Estas ações têm histórico de crescimento consistente!")

## Gráfico: Visualizando a Performance

In [ ]:
# Preparar dados para o gráfico
dados_grafico = top_5_performance.reset_index()
dados_grafico.columns = ['Ação', 'Variação Média']

# Criar figura
plt.figure(figsize=(12, 6))
barras = plt.bar(dados_grafico['Ação'], dados_grafico['Variação Média'],
                  color='#51CF66', alpha=0.8, edgecolor='black', linewidth=2)

# Adicionar valores nas barras
for barra in barras:
    altura = barra.get_height()
    plt.text(barra.get_x() + barra.get_width()/2., altura,
             f'{altura:.1f}%',
             ha='center', va='bottom', fontsize=11, fontweight='bold')

# Adicionar linha de referência
plt.axhline(y=0, color='black', linestyle='--', linewidth=1.5, alpha=0.7)

# Personalizar gráfico
plt.title('Top 5 Ações com Melhor Performance Média', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Código da Ação', fontsize=12, fontweight='bold')
plt.ylabel('Variação Média Diária (%)', fontsize=12, fontweight='bold')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()

# Salvar figura
plt.savefig(output_dir / '02_performance.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráfico salvo em: outputs/02_performance.png")

---

# 📊 ANÁLISE 3: VOLUME DE NEGOCIAÇÃO

## O que é Volume de Negociação?

Volume mede **quantas ações foram compradas e vendidas**. Ações com alto volume têm:
- ✅ **Maior liquidez** (fácil de comprar/vender)
- ✅ **Melhor preço** (menos variação de preço entre transações)
- ✅ **Menor risco** (mais seguras)

Ações com pouco volume são mais difíceis de negociar.

## Cálculo: Top 5 Ações Mais Negociadas

In [ ]:
# Calcular volume total para cada ação
volume_total = df.groupby('ticker')['volume'].sum()

# Selecionar as 5 ações mais negociadas
top_5_volume = volume_total.nlargest(5)

# Exibir resultados
print("🏆 TOP 5 AÇÕES MAIS NEGOCIADAS")
print("=" * 50)
for posicao, (acao, valor) in enumerate(top_5_volume.items(), 1):
    valor_bilhoes = valor / 1e9
    print(f"{posicao}. {acao}: {valor_bilhoes:.2f} bilhões de ações negociadas")

print("\n💡 Dica: Ações com alto volume são mais líquidas e seguras para negociar!")

## Gráfico: Visualizando o Volume

In [ ]:
# Preparar dados para o gráfico
dados_grafico = top_5_volume.reset_index()
dados_grafico.columns = ['Ação', 'Volume']
dados_grafico['Volume_bilhoes'] = dados_grafico['Volume'] / 1e9

# Criar figura
plt.figure(figsize=(12, 6))
barras = plt.bar(dados_grafico['Ação'], dados_grafico['Volume_bilhoes'],
                  color='#4ECDC4', alpha=0.8, edgecolor='black', linewidth=2)

# Adicionar valores nas barras
for barra in barras:
    altura = barra.get_height()
    plt.text(barra.get_x() + barra.get_width()/2., altura,
             f'{altura:.1f}B',
             ha='center', va='bottom', fontsize=11, fontweight='bold')

# Personalizar gráfico
plt.title('Top 5 Ações Mais Negociadas (Volume Total)', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Código da Ação', fontsize=12, fontweight='bold')
plt.ylabel('Volume (Bilhões de Ações)', fontsize=12, fontweight='bold')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()

# Salvar figura
plt.savefig(output_dir / '03_volume.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráfico salvo em: outputs/03_volume.png")

---

# 📋 RESUMO EXECUTIVO

In [ ]:
# Recalcular métricas para o resumo
volatilidade = df.groupby('ticker')['variacao_diaria'].std()
top_5_volateis = volatilidade.nlargest(5)

variacao_media = df.groupby('ticker')['variacao_diaria'].mean()
top_5_performance = variacao_media.nlargest(5)

volume_total = df.groupby('ticker')['volume'].sum()
top_5_volume = volume_total.nlargest(5)

# Estatísticas gerais
print("\n" + "="*70)
print("RESUMO EXECUTIVO - ANÁLISE DE MERCADO FINANCEIRO")
print("="*70)

print(f"\n📊 DADOS GERAIS:")
print(f"  • Total de registros analisados: {len(df):,}")
print(f"  • Número de ações diferentes: {df['ticker'].nunique()}")
print(f"  • Período: {df['data_id'].min()} até {df['data_id'].max()}")

print(f"\n📈 INDICADORES DO MERCADO:")
print(f"  • Volatilidade média: {df['variacao_diaria'].std():.2f}%")
print(f"  • Variação média diária: {df['variacao_diaria'].mean():.2f}%")
print(f"  • Volume médio diário: {df['volume'].mean():,.0f} ações")

print(f"\n🏆 PRINCIPAIS DESTAQUES:")
print(f"  • Ação mais volátil: {top_5_volateis.index[0]} ({top_5_volateis.iloc[0]:.2f}% de volatilidade)")
print(f"  • Melhor performance: {top_5_performance.index[0]} (+{top_5_performance.iloc[0]:.2f}% variação média)")
print(f"  • Mais negociada: {top_5_volume.index[0]} ({top_5_volume.iloc[0]/1e9:.2f} bilhões de ações)")

print(f"\n💡 RECOMENDAÇÕES:")
print(f"  🎯 Para investidores conservadores: escolher ações com volatilidade baixa")
print(f"  ⚡ Para investidores agressivos: considerar ações de alta volatilidade")
print(f"  📊 Para todos: priorizar ações com alto volume (melhor liquidez)")

print("\n" + "="*70)

---

# 🎯 ANÁLISE PROFUNDA: RECOMENDAÇÕES ESTRATÉGICAS

## Baseado nos Dados e Análises

In [ ]:
# Calcular métricas para recomendações
volatilidade = df.groupby('ticker')['variacao_diaria'].std()
variacao_media = df.groupby('ticker')['variacao_diaria'].mean()
volume_total = df.groupby('ticker')['volume'].sum()

# Criar score de recomendação
metricas = pd.DataFrame({
    'Volatilidade': volatilidade,
    'Variacao_Media': variacao_media,
    'Volume': volume_total
})

# Normalizar métricas (0-100)
metricas['Score_Volatilidade'] = 100 - (volatilidade / volatilidade.max() * 100)  # Menor volatilidade = melhor
metricas['Score_Performance'] = (variacao_media - variacao_media.min()) / (variacao_media.max() - variacao_media.min()) * 100
metricas['Score_Liquidez'] = (volume_total - volume_total.min()) / (volume_total.max() - volume_total.min()) * 100

# Criar recomendações
recomendacoes = {
    'Investidores Conservadores': {
        'Peso_Volatilidade': 0.5,
        'Peso_Performance': 0.3,
        'Peso_Liquidez': 0.2
    },
    'Investidores Moderados': {
        'Peso_Volatilidade': 0.35,
        'Peso_Performance': 0.35,
        'Peso_Liquidez': 0.3
    },
    'Investidores Agressivos': {
        'Peso_Volatilidade': 0.2,
        'Peso_Performance': 0.5,
        'Peso_Liquidez': 0.3
    }
}

print("\n" + "="*80)
print("🎯 CARTEIRAS RECOMENDADAS POR PERFIL DE INVESTIDOR")
print("="*80)

for perfil, pesos in recomendacoes.items():
    score = (metricas['Score_Volatilidade'] * pesos['Peso_Volatilidade'] +
             metricas['Score_Performance'] * pesos['Peso_Performance'] +
             metricas['Score_Liquidez'] * pesos['Peso_Liquidez'])
    
    top_3 = score.nlargest(3)
    
    print(f"\n{'='*80}")
    print(f"👤 {perfil.upper()}")
    print(f"{'='*80}")
    
    if perfil == 'Investidores Conservadores':
        print(f"""
📋 DESCRIÇÃO:
   Busca segurança e rentabilidade consistente. Prefere risco baixo.
   
🎯 CRITÉRIOS:
   • Volatilidade: 50% (menor volatilidade = mais seguro)
   • Performance: 30% (crescimento consistente)
   • Liquidez: 20% (facilidade de negociação)
   
💡 ESTRATÉGIA:
   • Investimentos de longo prazo
   • Busca por renda estável
   • Menor exposição a variações de preço
        """)
    
    elif perfil == 'Investidores Moderados':
        print(f"""
📋 DESCRIÇÃO:
   Equilibra segurança e retorno. Risco médio aceitável.
   
🎯 CRITÉRIOS:
   • Volatilidade: 35% (balance entre risco e retorno)
   • Performance: 35% (crescimento equilibrado)
   • Liquidez: 30% (segurança na negociação)
   
💡 ESTRATÉGIA:
   • Carteira diversificada
   • Investimentos de médio a longo prazo
   • Acompanhamento periódico
        """)
    
    else:
        print(f"""
📋 DESCRIÇÃO:
   Busca máxima rentabilidade. Aceita risco elevado.
   
🎯 CRITÉRIOS:
   • Volatilidade: 20% (maior volatilidade = maior retorno)
   • Performance: 50% (foco no crescimento)
   • Liquidez: 30% (mobilidade quando necessário)
   
💡 ESTRATÉGIA:
   • Investimentos de curto a médio prazo
   • Busca por ganhos rápidos
   • Monitoramento constante do mercado
        """)
    
    print(f"\n🏆 TOP 3 AÇÕES RECOMENDADAS:")
    for posicao, (acao, valor_score) in enumerate(top_3.items(), 1):
        vol = metricas.loc[acao, 'Volatilidade']
        perf = metricas.loc[acao, 'Variacao_Media']
        liq = metricas.loc[acao, 'Volume'] / 1e9
        print(f"   {posicao}. {acao}")
        print(f"      Score: {valor_score:.1f}/100")
        print(f"      Volatilidade: {vol:.2f}% | Performance: +{perf:.2f}% | Liquidez: {liq:.2f}B")

print("\n" + "="*80)

---

# 📊 ANÁLISE COMPARATIVA: RISCO vs RETORNO

In [ ]:
# Criar gráfico de dispersão Risco vs Retorno
volatilidade = df.groupby('ticker')['variacao_diaria'].std()
variacao_media = df.groupby('ticker')['variacao_diaria'].mean()
volume_total = df.groupby('ticker')['volume'].sum()

# Criar figura
plt.figure(figsize=(14, 8))

# Scatter plot com tamanho baseado no volume
scatter = plt.scatter(volatilidade, variacao_media, 
                      s=volume_total/1e7,  # Tamanho proporcional ao volume
                      alpha=0.6,
                      c=variacao_media,  # Cor baseada no retorno
                      cmap='RdYlGn',
                      edgecolors='black',
                      linewidth=1.5)

# Adicionar labels das ações
for acao in volatilidade.index:
    plt.annotate(acao, 
                (volatilidade[acao], variacao_media[acao]),
                fontsize=9,
                fontweight='bold',
                ha='center')

# Adicionar linhas de referência
plt.axhline(y=variacao_media.mean(), color='gray', linestyle='--', alpha=0.5, label='Média de Retorno')
plt.axvline(x=volatilidade.mean(), color='gray', linestyle='--', alpha=0.5, label='Média de Risco')

# Destacar quadrantes
plt.text(volatilidade.min() + 20, variacao_media.max() - 5, 'BAIXO RISCO\nALTO RETORNO\n(IDEAL)', 
        fontsize=10, fontweight='bold', bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))
plt.text(volatilidade.max() - 100, variacao_media.max() - 5, 'ALTO RISCO\nALTO RETORNO\n(AGRESSIVO)', 
        fontsize=10, fontweight='bold', bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.5))

# Personalizar gráfico
plt.title('Análise Risco vs Retorno - Todas as Ações', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Risco (Volatilidade %)', fontsize=12, fontweight='bold')
plt.ylabel('Retorno (Variação Média %)', fontsize=12, fontweight='bold')
plt.colorbar(scatter, label='Retorno Médio (%)')
plt.legend(loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()

# Salvar figura
plt.savefig(output_dir / '04_risco_vs_retorno.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráfico salvo em: outputs/04_risco_vs_retorno.png")

---

# 📄 GERAR RELATÓRIO EXECUTIVO COMPLETO

In [ ]:
# Preparar dados para o relatório
volatilidade = df.groupby('ticker')['variacao_diaria'].std()
top_5_volateis = volatilidade.nlargest(5)

variacao_media = df.groupby('ticker')['variacao_diaria'].mean()
top_5_performance = variacao_media.nlargest(5)

volume_total = df.groupby('ticker')['volume'].sum()
top_5_volume = volume_total.nlargest(5)

# Criar conteúdo do relatório
relatorio = f"""
╔════════════════════════════════════════════════════════════════════════════════╗
║                          RELATÓRIO EXECUTIVO                                  ║
║                  ANÁLISE COMPLETA DE MERCADO FINANCEIRO                       ║
║                        Com Recomendações Estratégicas                          ║
╚════════════════════════════════════════════════════════════════════════════════╝

📅 DATA DE GERAÇÃO: {pd.Timestamp.now().strftime('%d de %B de %Y às %H:%M')}
📊 PERÍODO ANALISADO: {df['data_id'].min()} até {df['data_id'].max()}

═══════════════════════════════════════════════════════════════════════════════════
1. RESUMO GERAL DOS DADOS
═══════════════════════════════════════════════════════════════════════════════════

📈 VOLUME DE DADOS:
   • Total de registros analisados: {len(df):,}
   • Número de ações diferentes: {df['ticker'].nunique()}
   • Dias de negociação: {df['data_id'].nunique()}
   
📊 INDICADORES PRINCIPAIS:
   • Volatilidade média do mercado: {df['variacao_diaria'].std():.2f}%
   • Variação média diária geral: {df['variacao_diaria'].mean():.2f}%
   • Volume médio diário: {df['volume'].mean():,.0f} ações
   • Preço médio de fechamento: R$ {df['preco_fechamento'].mean():.2f}

═══════════════════════════════════════════════════════════════════════════════════
2. TOP 5 AÇÕES MAIS VOLÁTEIS (MAIOR RISCO)
═══════════════════════════════════════════════════════════════════════════════════

Estas ações apresentam maior oscilação de preço. São ideais para investidores
agressivos que buscam maiores retornos em períodos curtos.

"""

for i, (acao, valor) in enumerate(top_5_volateis.items(), 1):
    relatorio += f"{i}. {acao}: {valor:.2f}% de volatilidade\n"

relatorio += f"""

═══════════════════════════════════════════════════════════════════════════════════
3. TOP 5 AÇÕES COM MELHOR PERFORMANCE (MAIOR RETORNO)
═══════════════════════════════════════════════════════════════════════════════════

Estas ações tiveram melhor histórico de crescimento. Indicam consistência
no crescimento de valor ao longo do período analisado.

"""

for i, (acao, valor) in enumerate(top_5_performance.items(), 1):
    relatorio += f"{i}. {acao}: +{valor:.2f}% de variação média diária\n"

relatorio += f"""

═══════════════════════════════════════════════════════════════════════════════════
4. TOP 5 AÇÕES MAIS NEGOCIADAS (MAIOR LIQUIDEZ)
═══════════════════════════════════════════════════════════════════════════════════

Esas ações têm maior volume de negociação. São mais fáceis de comprar/vender
e geralmente mais estáveis, pois possuem muitos compradores e vendedores.

"""

for i, (acao, valor) in enumerate(top_5_volume.items(), 1):
    relatorio += f"{i}. {acao}: {valor:,.0f} ações negociadas ({valor/1e9:.2f} bilhões)\n"

relatorio += f"""

═══════════════════════════════════════════════════════════════════════════════════
5. RECOMENDAÇÕES ESTRATÉGICAS POR PERFIL DE INVESTIDOR
═══════════════════════════════════════════════════════════════════════════════════

🎯 PERFIL 1: INVESTIDOR CONSERVADOR
────────────────────────────────────────────────────────────────────────────────

OBJETIVO: Segurança e rentabilidade consistente com risco mínimo.

CARACTERÍSTICAS:
• Prioriza estabilidade sobre ganhos máximos
• Investe pensando em longo prazo (5+ anos)
• Busca renda passiva segura
• Evita movimentos bruscos de preço

CRITÉRIOS DE SELEÇÃO:
1. Volatilidade BAIXA (abaixo de 100%)
2. Variação POSITIVA consistente
3. Volume ALTO (ações muito negociadas)

VANTAGENS:
✓ Menor risco de perdas significativas
✓ Previsibilidade melhor
✓ Ideal para aposentadoria e poupança
✓ Menor necessidade de monitoramento constante

DESVANTAGENS:
✗ Retornos menores em períodos curtos
✗ Menos oportunidades de ganho rápido
✗ Pode não acompanhar inflação em alguns períodos

AÇÕES RECOMENDADAS:
• Buscar entre ações com volatilidade < 100%
• Focar em ações com variação média > 80% da média geral
• Selecionar entre as mais negociadas (top 10 volume)

ALOCAÇÃO SUGERIDA:
• 60% em ações de baixa volatilidade e alta performance
• 30% em ações de alto volume para liquidez
• 10% em reserva para oportunidades


🎯 PERFIL 2: INVESTIDOR MODERADO
────────────────────────────────────────────────────────────────────────────────

OBJETIVO: Equilibrar segurança e retorno com risco moderado aceitável.

CARACTERÍSTICAS:
• Balanceia segurança e rentabilidade
• Investe em horizonte médio (3-5 anos)
• Aceita volatilidade moderada
• Monitora a carteira regularmente

CRITÉRIOS DE SELEÇÃO:
1. Volatilidade MODERADA (100-250%)
2. Variação POSITIVA acima da média
3. Volume MÉDIO a ALTO

VANTAGENS:
✓ Retornos melhores que perfil conservador
✓ Risco controlado e administrável
✓ Mais flexibilidade que conservador
✓ Permite diversificação estratégica

DESVANTAGENS:
✗ Risco moderado de perdas
✗ Requer monitoramento periódico
✗ Resultados podem ser inconsistentes

AÇÕES RECOMENDADAS:
• Misturar ações de diferentes volatilidades
• 50% em ações de performance comprovada
• 30% em ações de crescimento potencial
• 20% em ações de alta liquidez

ALOCAÇÃO SUGERIDA:
• 40% em ações estáveis
• 35% em ações de crescimento
• 25% em ações de alta liquidez


🎯 PERFIL 3: INVESTIDOR AGRESSIVO
────────────────────────────────────────────────────────────────────────────────

OBJETIVO: Maximizar retornos aceitando risco elevado.

CARACTERÍSTICAS:
• Busca máxima rentabilidade
• Investe em curto/médio prazo (1-3 anos)
• Aceita grande volatilidade
• Monitora constantemente o mercado
• Aproveita oportunidades rápidas

CRITÉRIOS DE SELEÇÃO:
1. Volatilidade ALTA (acima de 250%)
2. Variação POSITIVA mesmo com risco
3. Volume SUFICIENTE para entrar/sair rápido

VANTAGENS:
✓ Maior potencial de ganho em curto prazo
✓ Oportunidades de swing trading
✓ Aproveita picos de mercado
✓ Possível ganho rápido de capital

DESVANTAGENS:
✗ Risco muito elevado de perdas
✗ Requer monitoramento diário
✗ Necessita maior conhecimento de mercado
✗ Emocionalidade pode gerar erros

AÇÕES RECOMENDADAS:
• Focar em ações com volatilidade > 250%
• Misturar com algumas de performance consistente
• Manter posições líquidas para reagir rápido

ALOCAÇÃO SUGERIDA:
• 50% em ações voláteis de alta performance
• 30% em ações voláteis de potencial
• 20% em ações de menor volatilidade como "freio"


═══════════════════════════════════════════════════════════════════════════════════
6. DICAS IMPORTANTES PARA TODOS OS PERFIS
═══════════════════════════════════════════════════════════════════════════════════

⚠️  GESTÃO DE RISCO:
   1. NUNCA coloque todo o dinheiro em uma ação
   2. Diversifique em pelo menos 5-10 ações diferentes
   3. Defina um limite de perda aceitável (stop loss)
   4. Não invista dinheiro que você precise em curto prazo
   5. Mantenha sempre uma reserva de emergência

📊 MONITORAMENTO:
   1. Revise sua carteira pelo menos uma vez por mês
   2. Acompanhe notícias e eventos do mercado
   3. Rebalanceie conforme o mercado muda
   4. Registre todas as operações (para impostos e aprendizado)

💡 ESTRATÉGIAS:
   1. Compre ações em baixa, venda em alta
   2. Considere comprar periodicamente (investimento recorrente)
   3. Não venda por pânico em quedas temporárias
   4. Estude o mercado antes de investir
   5. Considere o longo prazo mesmo em curto prazo

🎯 ERROS A EVITAR:
   1. ❌ Seguir "dicas" de desconhecidos na internet
   2. ❌ Investir com dinheiro emprestado
   3. ❌ Tomar decisões emocionais
   4. ❌ Vender tudo por causa de uma notícia ruim
   5. ❌ Concentrar mais de 30% em uma ação
   6. ❌ Ignorar a volatilidade histórica


═══════════════════════════════════════════════════════════════════════════════════
7. PRÓXIMOS PASSOS
═══════════════════════════════════════════════════════════════════════════════════

1. Defina seu perfil de investidor (conservador, moderado ou agressivo)
2. Escolha 5-10 ações baseado nas recomendações acima
3. Comece com pequenos valores para aprender
4. Mantenha registros de suas operações
5. Revise este relatório mensalmente com dados novos
6. Busque educação financeira contínua
7. Considere consultar um assessor financeiro profissional


═══════════════════════════════════════════════════════════════════════════════════
FIM DO RELATÓRIO
═══════════════════════════════════════════════════════════════════════════════════

Grafos gerados:
  • 01_volatilidade.png - Ações mais voláteis
  • 02_performance.png - Melhor performance
  • 03_volume.png - Volume de negociação
  • 04_risco_vs_retorno.png - Análise completa

Próxima atualização recomendada: Sempre que houver novos dados disponíveis.
"""

# Salvar relatório
caminho_relatorio = output_dir / 'relatorio_completo.txt'
with open(caminho_relatorio, 'w', encoding='utf-8') as arquivo:
    arquivo.write(relatorio)

print("✅ Relatório completo salvo em: outputs/relatorio_completo.txt")
print("\n" + "="*80)
print("🎉 ANÁLISE COMPLETA COM SUCESSO!")
print("="*80)
print("\n📁 Arquivos gerados na pasta 'outputs':")
print("   ✓ 01_volatilidade.png")
print("   ✓ 02_performance.png")
print("   ✓ 03_volume.png")
print("   ✓ 04_risco_vs_retorno.png")
print("   ✓ relatorio_completo.txt")
print("\n📖 Abra 'relatorio_completo.txt' para ler as recomendações completas!")